In [403]:
import pandas as pd
import numpy as numpy
from geotext import GeoText
import re
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException

import spacy

import itertools
import requests

import gender_guesser.detector as gender

import warnings
warnings.filterwarnings('ignore')

#### Borial Management

In [89]:
url = "https://borialmanagement.com/"
res1 = requests.get(url)
sopa = BeautifulSoup(res1.content, 'html.parser')
tags = sopa.find_all("h4", {"class": "personName toBold"})

In [90]:
actores = []
for tag in tags:
    actores.append(tag.text)

In [91]:
actores.remove('GONZALO FERNГЃNDEZ')
actores.remove('VГЌCTOR CASTILLO')
actores.remove('ГЃLVARO QUINTANA')
actores.remove('CARLA DГЌAZ')
actores.remove('MARIO GARCГЌA')
actores.remove('SANTI MARГЌN')
actores.remove('ГЃLVARO FONTALBA')

In [92]:
actores2 = []
for actor in actores:
    if len(re.findall("\W", actor)) > 1:
        pass
    else:
        actores2.append(actor)

In [93]:
actores3 = []
for actor in actores2:
    actores3.append(actor.split())

In [94]:
d = gender.Detector()

nombres_por_genero = {"masculino": [], "femenino": []}

for nombre in actores3:
    nombre_completo = nombre[0].capitalize() + nombre[1].capitalize()
    genero = d.get_gender(nombre[0].capitalize())
    if genero == "male":
        nombres_por_genero["masculino"].append(nombre_completo)
    elif genero == "female":
        nombres_por_genero["femenino"].append(nombre_completo)

In [8]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

dicc = {"actor": [], "formacion": []}

for k, v in nombres_por_genero.items():
    if k == "masculino":
        for k in range(len(v)):
            url = f"https://borialmanagement.com/actores/{nombres_por_genero['masculino'][k]}"
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(url)
            driver.implicitly_wait(10)
            resultado = driver.find_element("css selector", "body > div:nth-child(3) > div > div:nth-child(2) > div").text
            resultado = resultado.split("\n")
            if resultado[1] == "FORMACIÓN":
                artista = resultado[0].lower()
                resultado.pop(0)
                resultado.pop(0)
                i = 0
                while resultado[i] != "TELEVISIÓN":
                    dicc["formacion"].append(resultado[i])
                    dicc["actor"].append(artista)
                    i+=1
            else:
                dicc["formacion"].append("no hay formación")
                dicc["actor"].append(artista)
    else:
        for k in range(len(v)):
            url = f"https://borialmanagement.com/actrices/{nombres_por_genero['femenino'][k]}"
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(url)
            driver.implicitly_wait(10)
            resultado = driver.find_element("css selector", "body > div:nth-child(3) > div > div:nth-child(2) > div").text
            resultado = resultado.split("\n")
            if resultado[1] == "FORMACIÓN":
                artista = resultado[0].lower()
                resultado.pop(0)
                resultado.pop(0)
                i = 0
                while (resultado[i] != "TELEVISIÓN") and (resultado[i] != "TEATRO") and (resultado[i] != "CINE"):
                    print(i)
                    dicc["formacion"].append(resultado[i])
                    dicc["actor"].append(artista)
                    i+=1
            else:
                dicc["formacion"].append("no hay formación")
                dicc["actor"].append(artista)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
0
1
2
3
4
5
6
7
8
9
10
11
12
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [9]:
df_borial = pd.DataFrame(dicc)

In [37]:
def extraer_ciudad(texto):
    if type(texto) != str:
        pass
    else:
        return GeoText(texto).cities

In [61]:
df_borial["ciudad"] = df_borial["formacion"].apply(extraer_ciudad)

In [41]:
def nombres(texto):
    if type(texto) != str:
        pass
    else:
        patron = r'[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+\s[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+'
        return re.findall(patron, texto)

In [202]:
def sininfo(valor):
    if valor is None:
        return "no hay info"
    elif isinstance(valor, list) and len(valor) > 0:
        return valor[0]
    elif len(valor) == 0:
        return "no hay info"
    else:
        return valor

In [151]:
df_borial["nombres"] = df_borial["formacion"].apply(nombres)

In [160]:
df_borial = df_borial.explode("nombres")

In [163]:
df_borial.to_csv("../data/borial_data")

In [200]:
df_borial = pd.read_csv("../data/borial_data", index_col=0)

In [203]:
df_borial["ciudad"] = df_borial["ciudad"].apply(sininfo)

In [217]:
def lista(x):
    if type(x) == list:
        return str(x[0])
    else:
        return x

In [218]:
df_borial["ciudad"] = df_borial["ciudad"].apply(lista)

In [223]:
df_borial.head(20)

,actor,formacion,ciudad,nombres
0,daniel albaladejo,Licenciado en Interpretación por la E.S.A.D. d...,['Murcia'],NaN
1,daniel albaladejo,Cursos de perfeccionamiento con:,[],NaN
2,daniel albaladejo,Vicente Fuentes (Siglo de Oro).,[],Vicente Fuentes
3,daniel albaladejo,Will Keen (Shakespeare).,[],Will Keen
4,daniel albaladejo,Francisco Torres Monreal (Teatro Contemporáneo).,[],Francisco Torres
4,daniel albaladejo,Francisco Torres Monreal (Teatro Contemporáneo).,[],Teatro Contemporáneo
5,daniel albaladejo,Norman Taylor (Expresión Dramática y Movimiento).,[],Norman Taylor
5,daniel albaladejo,Norman Taylor (Expresión Dramática y Movimiento).,[],Expresión Dramática
6,daniel albaladejo,Antón Valén (Técnicas de Clown y Bufón).,[],Antón Valén
7,enrique asenjo,"Dramaturgia Teatral, impartido por Sanchís Sin...",[],Dramaturgia Teatral


In [224]:
df_borial['nº formaciones'] = df_borial['actor'].apply(lambda x: df_borial[df_borial['actor'] == x]['actor'].value_counts().values[0])

In [231]:
df_borial.drop("formacion", axis=1, inplace=True)

In [237]:
df_borial.to_csv("../data/borial_data_good.csv")

#### Única Representaciones

In [95]:
url = "https://www.unicarepresentaciones.com/Actores"
res1 = requests.get(url)
sopa = BeautifulSoup(res1.content, 'html.parser')
tags = sopa.find_all("div", {"class": "views-field views-field-title"})

In [96]:
h_unica = []
for tag in tags:
    h_unica.append(tag.text)

In [97]:
url = "https://www.unicarepresentaciones.com/Actrices"
res1 = requests.get(url)
sopa = BeautifulSoup(res1.content, 'html.parser')
tags = sopa.find_all("div", {"class": "views-field views-field-title"})

In [98]:
m_unica = []
for tag in tags:
    m_unica.append(tag.text)

In [99]:
actores_unica = h_unica+m_unica

In [100]:
h_unica_scrap = []
for actor in (h_unica):
    h_unica_scrap.append(actor.strip().replace(" ", "-").lower())

In [101]:
m_unica_scrap = []
for actriz in (m_unica):
    m_unica_scrap.append(actriz.strip().replace(" ", "-").lower())

In [102]:
unica_scrap = h_unica_scrap+m_unica_scrap

In [172]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized')
opciones.add_argument('user.data-dir=selenium')
opciones.add_argument('--incognito')

dicc = {"actor": [], "formacion": []}

for actor in unica_scrap:
    try:
        url = f"https://www.unicarepresentaciones.com/actores-actrices/{actor}"
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)
        driver.implicitly_wait(10)
        cv = driver.find_element("css selector", "#block-system-main > div > div.ficha-actores > div.ficha-col-1").text
        elementos = cv.split("\n")
        elementos.pop(0)
        elementos = [i.strip() for i in elementos]
        if "Formación" not in elementos:
            pass
        else:
            indice_formacion = elementos.index("Formación") + 1
            lista_formacion = elementos[indice_formacion:]

            dicc["actor"].append(actor)
            dicc["formacion"].append(lista_formacion)
    except NoSuchElementException:
        driver.close()

In [180]:
df_unica = pd.DataFrame(dicc)

In [181]:
df_unica = df_unica.explode('formacion')

In [177]:
df_unica.to_csv("../data/unica_data_raw")

In [170]:
df_unica = pd.read_csv("../data/unica_data_raw", index_col=0)

In [171]:
df_unica["ciudad"] = df_unica["formacion"].apply(extraer_ciudad)

In [173]:
df_unica["nombres"] = df_unica["formacion"].apply(nombres)

In [174]:
df_unica['nº formaciones'] = df_unica['actor'].apply(lambda x: df_unica[df_unica['actor'] == x]['actor'].value_counts().values[0])

In [175]:
df_unica = df_unica.explode("nombres")

In [180]:
df_unica['ciudad'] = df_unica['ciudad'].apply(sininfo)

In [234]:
df_unica.drop("formacion", axis=1, inplace=True)

In [236]:
df_unica.to_csv("../data/unica_data_good.csv")

In [434]:
df_a = pd.read_csv("../data/borial_data_good.csv", index_col=0)
df_a["representante"] = "Borial Management"
df_b =pd.read_csv("../data/unica_data_good.csv", index_col=0)
df_b["representante"] = "Única Representaciones"
df = pd.concat([df_a, df_b], ignore_index=True)

,actor,ciudad,nombres,nº formaciones,representante
0,daniel albaladejo,['Murcia'],NaN,9,Borial Management
1,daniel albaladejo,[],NaN,9,Borial Management
2,daniel albaladejo,[],Vicente Fuentes,9,Borial Management
3,daniel albaladejo,[],Will Keen,9,Borial Management
4,daniel albaladejo,[],Francisco Torres,9,Borial Management
...,...,...,...,...,...
998,mar-vidal,no hay info,NaN,20,Única Representaciones
999,mar-vidal,no hay info,NaN,20,Única Representaciones
1000,mar-vidal,Cartagena,NaN,20,Única Representaciones
1001,mar-vidal,no hay info,NaN,20,Única Representaciones


In [416]:
df = pd.read_csv("../data/repre_data.csv", index_col=0)

In [436]:
df.drop(["nº formaciones"], axis=1, inplace=True)

In [437]:
df.to_csv("../data/repre_data.csv")

In [420]:
df_formaciones = df.drop(["ciudad", "nombres"], axis=1)

In [425]:
df_formaciones.drop_duplicates(keep='first', inplace=True)

In [426]:
df_formaciones.to_csv("../data/formaciones.csv")

#### Listado IMDB

In [253]:
listado_actores = actores2 + actores_unica
len(listado_actores)

75

In [397]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized')
opciones.add_argument('user.data-dir=selenium')
opciones.add_argument('--incognito')

dicc = {"actor": [], "titulo": [], "tipo": [], "año": [], "episodios": []}

desplazamiento = [2400, 2500, 2600, 2700, 2800, 2900]
for actor in listado_actores:
    url = "https://www.imdb.com/?ref_=nv_home"
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    driver.implicitly_wait(10)
    element = driver.find_element("css selector", "#suggestion-search").send_keys(actor)
    driver.find_element("css selector", "#react-autowhatever-navSuggestionSearch--item-0 > a > div.sc-fEXmlR.iAJXhE > div.sc-bjfHbI.fOUGiO.searchResult__constTitle").click()
    driver.implicitly_wait(10)
    for i in desplazamiento:
        try:
            driver.execute_script(f"window.scrollTo(0, {i})")
            driver.implicitly_wait(10)
            sleep(5)
            driver.find_element("css selector", "#iconContext-expand-more").click()
            driver.implicitly_wait(10)
            conteo+=1
            num_titulos = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-f9e7f53-0.ifXVtO > div > section > div > div.sc-414674b4-1.gWfYnM.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(2) > div.sc-4390696d-3.dVbZNr > div.ipc-accordion.sc-3aa8a953-0.bVYxxN.date-credits-accordion.ipc-accordion--base.ipc-accordion--dividers-none.ipc-accordion--pageSection > div > div.ipc-accordion__item__header.ipc-accordion__item__header--sticky > label > span.ipc-accordion__item__title > ul > li.ipc-inline-list__item.credits-total").text
            num_titulos = int(num_titulos)
            for i in range(1, num_titulos):
                dicc["actor"].append(actor)
                dicc["titulo"].append(driver.find_element("css selector",f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-f9e7f53-0.ifXVtO > div > section > div > div.sc-414674b4-1.gWfYnM.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(2) > div.sc-4390696d-3.dVbZNr > div.ipc-accordion.sc-3aa8a953-0.bVYxxN.date-credits-accordion.ipc-accordion--base.ipc-accordion--dividers-none.ipc-accordion--pageSection > div > div.ipc-accordion__item__content > div > ul > li:nth-child({i}) > div.ipc-metadata-list-summary-item__c > div.ipc-metadata-list-summary-item__tc > a").text)
                try:
                    dicc["episodios"].append(driver.find_element("css selector",f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-f9e7f53-0.ifXVtO > div > section > div > div.sc-414674b4-1.gWfYnM.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(2) > div.sc-4390696d-3.dVbZNr > div.ipc-accordion.sc-3aa8a953-0.bVYxxN.date-credits-accordion.ipc-accordion--base.ipc-accordion--dividers-none.ipc-accordion--pageSection > div > div.ipc-accordion__item__content > div > ul > li:nth-child({i}) > div.ipc-metadata-list-summary-item__c > div.ipc-metadata-list-summary-item__cc > ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--no-wrap.ipc-inline-list--inline.ipc-metadata-list-summary-item__cbl.base > li > button").text)
                    dicc["año"].append(driver.find_element("css selector",f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-f9e7f53-0.ifXVtO > div > section > div > div.sc-414674b4-1.gWfYnM.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(2) > div.sc-4390696d-3.dVbZNr > div.ipc-accordion.sc-3aa8a953-0.bVYxxN.date-credits-accordion.ipc-accordion--base.ipc-accordion--dividers-none.ipc-accordion--pageSection > div > div.ipc-accordion__item__content > div > ul > li:nth-child({i}) > div.ipc-metadata-list-summary-item__c > div.ipc-metadata-list-summary-item__cc > ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--no-wrap.ipc-inline-list--inline.ipc-metadata-list-summary-item__ctl.base > li > span").text)
                except NoSuchElementException:
                    dicc["año"].append(driver.find_element("css selector",f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-f9e7f53-0.ifXVtO > div > section > div > div.sc-414674b4-1.gWfYnM.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(2) > div.sc-4390696d-3.dVbZNr > div.ipc-accordion.sc-3aa8a953-0.bVYxxN.date-credits-accordion.ipc-accordion--base.ipc-accordion--dividers-none.ipc-accordion--pageSection > div > div.ipc-accordion__item__content > div > ul > li:nth-child({i}) > div.ipc-metadata-list-summary-item__c > div.ipc-metadata-list-summary-item__cc > ul > li > span").text)
                    dicc["episodios"].append("no hay info")
                try:
                    dicc["tipo"].append(driver.find_element("css selector",f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-f9e7f53-0.ifXVtO > div > section > div > div.sc-414674b4-1.gWfYnM.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(2) > div.sc-4390696d-3.dVbZNr > div.ipc-accordion.sc-3aa8a953-0.bVYxxN.date-credits-accordion.ipc-accordion--base.ipc-accordion--dividers-none.ipc-accordion--pageSection > div > div.ipc-accordion__item__content > div > ul > li:nth-child({i}) > div.ipc-metadata-list-summary-item__c > div.ipc-metadata-list-summary-item__tc > ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--no-wrap.ipc-inline-list--inline.ipc-metadata-list-summary-item__stl.base").text)
                except NoSuchElementException:
                    dicc["tipo"].append("no hay info")

        except (ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException):
            pass

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:01<00:00, 4.36MB/s]


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=112.0.5615.86)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00CADCE3+50899]
	(No symbol) [0x00C3E111]
	(No symbol) [0x00B45450]
	(No symbol) [0x00B71846]
	(No symbol) [0x00B694A5]
	(No symbol) [0x00B8B8FC]
	(No symbol) [0x00B68EC6]
	(No symbol) [0x00B8BC54]
	(No symbol) [0x00B9E01C]
	(No symbol) [0x00B8B6F6]
	(No symbol) [0x00B67708]
	(No symbol) [0x00B6886D]
	GetHandleVerifier [0x00F13EAE+2566302]
	GetHandleVerifier [0x00F492B1+2784417]
	GetHandleVerifier [0x00F4327C+2759788]
	GetHandleVerifier [0x00D45740+672048]
	(No symbol) [0x00C48872]
	(No symbol) [0x00C441C8]
	(No symbol) [0x00C442AB]
	(No symbol) [0x00C371B7]
	BaseThreadInitThunk [0x75F37D69+25]
	RtlInitializeExceptionChain [0x7707B74B+107]
	RtlClearBits [0x7707B6CF+191]


In [399]:
df_trabajos = pd.DataFrame(dicc)
# df_trabajos["trabajos"] = df_trabajos["trabajos"].str.split("\n")
df_trabajos

,actor,titulo,tipo,año,episodios
0,DANIEL ALBALADEJO,Alumbramiento,Corto,2022,no hay info
1,DANIEL ALBALADEJO,La noche más larga,Miniserie de TV,2022,6 episodios
2,DANIEL ALBALADEJO,Cinco estrellas,Corto,2021,no hay info
3,DANIEL ALBALADEJO,El Cid,Serie de TV,2020–2021,10 episodios
4,DANIEL ALBALADEJO,Oculto,Corto,2021,no hay info
...,...,...,...,...,...
754,Juanma Cifuentes,Menos es más,no hay info,2000,no hay info
755,Juanma Cifuentes,El súper,Serie de TV,1999,1 episodio
756,Juanma Cifuentes,Manos a la obra,Serie de TV,1999,1 episodio
757,Juanma Cifuentes,La casa de los líos,Serie de TV,1998,1 episodio


In [405]:
df_trabajos.to_csv("../data/imdb.csv")